In [1]:
import datetime
from binance import Client
import pandas as pd
import numpy as np

# Binance client : client = Client(API_number, secret_key)

class BinanceAPI:
    
    # convert_timestamp converts a timestamp into a date 
    @staticmethod
    def convert_timestamp(timestamp):
        if (timestamp / 10**13) > 1:
            print("Careful, the timestamp returned by the API is in milliseconds")

        date = datetime.datetime.fromtimestamp(timestamp)
        return(date)
    
    # get_tickers gets every possible crypto ticker provided by the API 
    @staticmethod
    def get_tickers(client):
        # Get tickers from client
        all_tickers = client.get_all_tickers()
        tickers_usdt = []
    
        # Only return tickers that end with "USDT"
        for k in range(len(all_tickers)):
            if all_tickers[k]['symbol'][-4:] == 'USDT':
                tickers_usdt.append(all_tickers[k]['symbol'])
        return(tickers_usdt)
    
    # Testing wether or not a ticker is available from a certain date
    @staticmethod
    def test_get_ticker(client, ticker, interval, number_days):    
        try:
            # Testing the get price for the first day of data we have to get
            format_str = 'day ago UTC'
            start_str = str(number_days) + format_str
            end_str = str(number_days - 1) + format_str

            # Getting klines from the API
            klines = client.get_historical_klines(ticker, interval = interval, start_str = start_str, end_str = end_str)
        except:
            print("get_historical_klines method failed for ticker {}".format(ticker))
    
    # Testing the tickers of a given list of tickers
    @staticmethod
    def test_tickers(client, tickers, number_days):
        for ticker in tickers:
            test_get_ticker(ticker, number_days)
    
    # Getting historical prices for given tickers, number of days, interval. Returns a pandas dataframe
    @staticmethod
    def get_ticker_data(client, ticker, number_days, interval):
        # Getting klines
        start_str = str(number_days) + 'day ago UTC'
        klines = client.get_historical_klines(ticker, interval = interval, start_str = start_str)

        dates, close_prices = [], []
        for line in klines:
            # timestamp returned by the API is in milliseconds, thus dividing by 1000 before passing it into the converter
            timestamp_seconds = (line[0] / 10**3)

            # appending the date and the close price in the right lists
            dates.append(str(BinanceAPI.convert_timestamp(timestamp_seconds))[:10])
            close_prices.append(float(line[4]))

        # Appending prices in a dataframe
        data = {ticker : close_prices}
        df_return = pd.DataFrame(data = data)
        df_return.index = dates
        return(df_return)
    
    @staticmethod
    def get_data(client, tickers, number_days, interval):
        # Creating an empty list that we will append dfs in
        list_dfs = []

        for ticker in tickers:
            df_ticker = BinanceAPI.get_ticker_data(ticker, number_days, interval)
            list_dfs.append(df_ticker)

        # Merging dataframes
        df_return = pd.concat([df for df in list_dfs], axis=1)
        return(df_return)

In [2]:
log = 'API_KEY'
key = 'SECRET_KEY'

In [3]:
client = Client(log,key)

In [4]:
BinanceAPI.get_ticker_data(client, 'BTCUSDT', 1, '1d')

,BTCUSDT
2021-10-14,57998.97
